# new code

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install plotly.express

Note: you may need to restart the kernel to use updated packages.


In [5]:
ls

LFMS.ipynb              food_listings_data.csv  receivers_data.csv
claims_data.csv         providers_data.csv


# Load datasets

In [6]:
receivers = pd.read_csv('receivers_data.csv')
food_listings = pd.read_csv('food_listings_data.csv')
claims = pd.read_csv('claims_data.csv')

@st.cache_data

def load_data():
    providers = pd.read_csv('providers_data.csv')
    receivers = pd.read_csv('receivers_data.csv')
    food_listings = pd.read_csv('food_listings_data.csv')
    claims = pd.read_csv('claims_data.csv')
    return providers, receivers, food_listings, claims 


2025-12-16 12:59:14.210 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


 # Data cleaning functions

In [7]:
def clean_data(providers, receivers, food_listings, claims):
    # Clean providers
    providers['City'] = providers['City'].str.title()
    providers['Contact'] = providers['Contact'].astype(str)

# Clean receivers

In [8]:
receivers['City'] = receivers['City'].str.title()
receivers['Contact'] = receivers['Contact'].astype(str)

 # Clean food listings

In [9]:
food_listings['Expiry_Date'] = pd.to_datetime(food_listings['Expiry_Date'], errors='coerce')
food_listings['Location'] = food_listings['Location'].str.title()
food_listings['Quantity'] = pd.to_numeric(food_listings['Quantity'], errors='coerce').fillna(0)

 # Clean claims

In [10]:
claims['Timestamp'] = pd.to_datetime(claims['Timestamp'], errors='coerce')


In [13]:
providers, receivers, food_listings, claims = load_data()

2025-12-16 16:18:40.846 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-12-16 16:18:40.854 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-16 16:18:41.007 
  command:

    streamlit run /Users/OpenWorld/Documents/Project2/Pro2/.venv/lib/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-12-16 16:18:41.007 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-16 16:18:41.008 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-16 16:18:41.008 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-16 16:18:41.029 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-16 16:18

In [14]:
print("✅ Data loaded and cleaned successfully!")

✅ Data loaded and cleaned successfully!


# Create SQLite database

In [20]:
conn = sqlite3.connect('food_wastage.db')

def create_tables():
    # Providers table
    providers.to_sql('providers', conn, if_exists='replace', index=False)
    
    # Receivers table
    receivers.to_sql('receivers', conn, if_exists='replace', index=False)
    
    # Food listings table
    food_listings.to_sql('food_listings', conn, if_exists='replace', index=False)
    
    # Claims table
    claims.to_sql('claims', conn, if_exists='replace', index=False)
    
    print("✅ Database tables created successfully!")

create_tables()

✅ Database tables created successfully!


 # Complete SQL Queries (15+ Queries)

In [24]:
def execute_all_queries():
    queries = {
        "Q1: Providers & Receivers per City": """
        SELECT 
    COALESCE(p.City, r.City) AS City,
    COUNT(DISTINCT p.Provider_ID) AS Total_Providers,
    COUNT(DISTINCT r.Receiver_ID) AS Total_Receivers
FROM providers p
LEFT JOIN receivers r ON p.City = r.City
GROUP BY COALESCE(p.City, r.City)
ORDER BY Total_Providers DESC
        """,
        
        "Q2: Provider Type Contribution": """
        SELECT 
    fl.Location AS City,
    SUM(fl.Quantity) AS Total_Supply
FROM food_listings fl
LEFT JOIN receivers r ON fl.Location = r.City
GROUP BY fl.Location
ORDER BY Total_Supply DESC
        """,
        
        "Q3: Providers in Specific City": """
        SELECT Name, Type, Address, Contact
FROM providers
WHERE LOWER(TRIM(City)) = LOWER(TRIM('Pune'));
        """,
        
        "Q4: Top Receivers by Claims": """
        SELECT r.Name, r.Type, r.City, COUNT(*) as Total_Claims
        FROM claims c
        JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
        GROUP BY r.Receiver_ID, r.Name, r.Type, r.City
        ORDER BY Total_Claims DESC
        LIMIT 10
        """,
        
        "Q5: Total Food Quantity Available": """
        SELECT SUM(Quantity) as Total_Food_Quantity
        FROM food_listings
        WHERE Expiry_Date > CURRENT_DATE
        """,
        
        "Q6: City with Most Food Listings": """
        SELECT Location as City, COUNT(*) as Total_Listings
        FROM food_listings
        GROUP BY Location
        ORDER BY Total_Listings DESC
        LIMIT 5
        """,
        
        "Q7: Most Common Food Types": """
        SELECT Food_Type, COUNT(*) as Count, SUM(Quantity) as Total_Quantity
        FROM food_listings
        GROUP BY Food_Type
        ORDER BY Total_Quantity DESC
        """,
        
        "Q8: Claims per Food Item": """
        SELECT fl.Food_Name, COUNT(*) as Total_Claims
        FROM claims c
        JOIN food_listings fl ON c.Food_ID = fl.Food_ID
        GROUP BY fl.Food_ID, fl.Food_Name
        ORDER BY Total_Claims DESC
        LIMIT 10
        """,
        
        "Q9: Provider with Most Successful Claims": """
        SELECT p.Name, p.City, COUNT(*) as Successful_Claims
        FROM claims c
        JOIN food_listings fl ON c.Food_ID = fl.Food_ID
        JOIN providers p ON fl.Provider_ID = p.Provider_ID
        WHERE c.Status = 'Completed'
        GROUP BY p.Provider_ID, p.Name, p.City
        ORDER BY Successful_Claims DESC
        LIMIT 5
        """,
        
        "Q10: Claim Status Percentage": """
        SELECT Status, 
            COUNT(*) as Count,
               ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM claims), 2) as Percentage
        FROM claims
        GROUP BY Status
        """,
        
        "Q11: Avg Quantity per Receiver": """
        SELECT AVG(claimed_quantity) as Avg_Quantity_Per_Receiver
        FROM (
            SELECT c.Receiver_ID, SUM(fl.Quantity) as claimed_quantity
            FROM claims c
            JOIN food_listings fl ON c.Food_ID = fl.Food_ID
            WHERE c.Status = 'Completed'
            GROUP BY c.Receiver_ID
        )
        """,
        
        "Q12: Most Claimed Meal Type": """
        SELECT Meal_Type, COUNT(*) as Total_Claims
        FROM claims c
        JOIN food_listings fl ON c.Food_ID = fl.Food_ID
        GROUP BY Meal_Type
        ORDER BY Total_Claims DESC
        """,
        
        "Q13: Total Quantity by Provider": """
        SELECT p.Name, p.Type, SUM(fl.Quantity) as Total_Donated
        FROM providers p
        JOIN food_listings fl ON p.Provider_ID = fl.Provider_ID
        GROUP BY p.Provider_ID, p.Name, p.Type
        ORDER BY Total_Donated DESC
        LIMIT 10
        """,
        
        "Q14: Food Available by Expiry": """
        SELECT 
            CASE 
                WHEN Expiry_Date > DATE('now', '+7 days') THEN 'More than 7 days'
                WHEN Expiry_Date > DATE('now') THEN 'Within 7 days'
                ELSE 'Expired'
            END as Expiry_Category,
            COUNT(*) as Count,
            SUM(Quantity) as Total_Quantity
        FROM food_listings
        GROUP BY 1
        ORDER BY Total_Quantity DESC
        """,
        
        "Q15: City-wise Demand vs Supply": """
        SELECT 
            COALESCE(fl.Location, r.City) as City,
            COALESCE(SUM(fl.Quantity), 0) as Total_Supply,
            COUNT(DISTINCT c.Claim_ID) as Total_Demand
        FROM food_listings fl
        FULL OUTER JOIN claims c ON fl.Location = r.City
        FULL OUTER JOIN receivers r ON fl.Location = r.City
        GROUP BY City
        ORDER BY Total_Supply DESC
        """
    }
    
    results = {}
    for name, query in queries.items():
        try:
            df = pd.read_sql_query(query, conn)
            results[name] = df
            print(f"✅ {name}")
            print(df.head())
            print("-" * 50)
        except Exception as e:
            print(f"❌ Error in {name}: {str(e)}")
    
    return results

# Execute all queries
query_results = execute_all_queries()


✅ Q1: Providers & Receivers per City
                       City  Total_Providers  Total_Receivers
0  South Christopherborough                3                0
1                 New Carol                3                0
2               Williamview                2                0
3         West Lauraborough                2                0
4          West Christopher                2                1
--------------------------------------------------
✅ Q2: Provider Type Contribution
            City  Total_Supply
0  South Kathryn           179
1   Jonathanstad           169
2      New Carol           167
3    North Keith           158
4      Jimmyberg           150
--------------------------------------------------
✅ Q3: Providers in Specific City
Empty DataFrame
Columns: [Name, Type, Address, Contact]
Index: []
--------------------------------------------------
✅ Q4: Top Receivers by Claims
                Name        Type        City  Total_Claims
0       Scott Hunter  Individua

 # Streamlit Application (Complete UI)